# EBAC - Regressão II - regressão múltipla

## Tarefa II

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [158]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Lasso
import statsmodels.api as sm
import numpy as np
import statsmodels.formula.api as smf
from sklearn.tree import DecisionTreeRegressor

In [96]:
df = pd.read_csv('previsao_de_renda.csv')

In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [98]:
# Remova as colunas que não são variáveis preditoras (se necessário)
df = df.drop(['Unnamed: 0', 'data_ref', 'id_cliente'], axis=1)

In [99]:
df = df.dropna()

In [100]:
df.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [101]:
# Substitua 'sua_coluna_alvo' pelo nome da coluna que você está tentando prever
target_column = 'renda'

# Selecionar as variáveis categóricas
categorical_columns = ['sexo', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia']

# Criar variáveis dummy para as variáveis categóricas
df_2 = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

df_2.head()


,posse_de_veiculo,posse_de_imovel,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia,renda,sexo_M,tipo_renda_Bolsista,tipo_renda_Empresário,...,educacao_Superior incompleto,estado_civil_Separado,estado_civil_Solteiro,estado_civil_União,estado_civil_Viúvo,tipo_residencia_Casa,tipo_residencia_Com os pais,tipo_residencia_Comunitário,tipo_residencia_Estúdio,tipo_residencia_Governamental
0,False,True,0,26,6.602740,1.0,8060.34,0,0,1,...,0,0,1,0,0,1,0,0,0,0
1,True,True,0,28,7.183562,2.0,1852.15,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,True,True,0,35,0.838356,2.0,2253.89,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,False,True,1,30,4.846575,3.0,6600.77,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,True,False,0,33,4.293151,1.0,6475.97,1,0,0,...,0,0,1,0,0,0,0,0,0,1


In [102]:
df.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [147]:
#1

X = df_2.drop('renda', axis=1)  # Features
y = df_2['renda']  # Variável alvo

# Dividir o conjunto de dados em treinamento e teste (75% para treinamento, 25% para teste)
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, test_size=0.25, random_state=42)

# Agora, X_train e y_train contêm os dados de treinamento, e X_test e y_test contêm os dados de teste

In [148]:
# Defina os valores de alpha que você quer testar
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

# Loop sobre os valores de alpha
for alpha in alphas:
    # Crie um modelo de regressão Ridge
    ridge_model = Ridge(alpha=alpha)

    # Treine o modelo com os dados de treinamento
    ridge_model.fit(X_train_1, y_train_1)

    # Faça previsões nos dados de teste
    y_pred = ridge_model.predict(X_test_1)

    # Avalie o R^2
    r2 = r2_score(y_test_1, y_pred)

    # Exiba o resultado
    print(f'R^2 para alpha={alpha}: {r2}')

R^2 para alpha=0: 0.2979664017691027
R^2 para alpha=0.001: 0.2979664666220776
R^2 para alpha=0.005: 0.29796672589706097
R^2 para alpha=0.01: 0.29796704968324395
R^2 para alpha=0.05: 0.29796962777106295
R^2 para alpha=0.1: 0.2979728203542503


O modelo com alpha = 0.1 tem o maior R², logo tende a ser o melhor modelo.

R² = 0.2979728203542503

In [151]:
# Criar modelos LASSO para diferentes valores de alpha e avaliar o R² na base de testes

#Loop sobre os valores de alpha
for alpha in alphas:
    # Crie um modelo de regressão LASSO
    lasso_model = Lasso(alpha=alpha)

    # Treine o modelo com os dados de treinamento
    lasso_model.fit(X_train_1, y_train_1)

    # Faça previsões nos dados de teste
    y_pred = lasso_model.predict(X_test_1)

    # Avalie o R^2
    r2 = r2_score(y_test_1, y_pred)

    # Exiba o resultado
    print(f'R^2 para alpha={alpha}: {r2}')

<ipython-input-151-30ccf9afe7e1>:9: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lasso_model.fit(X_train_1, y_train_1)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.036e+11, tolerance: 8.060e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


R^2 para alpha=0: 0.2979667742437567


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.876e+11, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(


R^2 para alpha=0.001: 0.29796724261602436


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.343e+11, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(


R^2 para alpha=0.005: 0.29796911346591415


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.858e+11, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.001e+10, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(


R^2 para alpha=0.01: 0.29797144609011794
R^2 para alpha=0.05: 0.29798986955710816
R^2 para alpha=0.1: 0.29801230507519516


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.780e+10, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(


O método Lasso com alpha = 0.1 obteve o maior R².

R² para alpha=0.1: 0.29801230507519516

In [106]:
#Teste com stepwise

def stepwise_selection(X, y,
                       initial_list=[],
                       threshold_in=0.05,
                       threshold_out = 0.05,
                       verbose=True):
    """ Perform a forward-backward feature selection
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [126]:
#1

# Features (X) e Variável alvo (y)
X = df.drop('renda', axis=1)  # Exclui a coluna 'renda'
y = df['renda']

# Dividir o conjunto de dados em treinamento e teste (75% para treinamento, 25% para teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [127]:
X_train_a = X_train.dropna()
X_train_a

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia
10766,F,True,True,0,Assalariado,Superior completo,Casado,Casa,50,5.509589,2.0
936,M,False,True,1,Assalariado,Secundário,Casado,Casa,23,1.158904,3.0
695,M,False,False,0,Servidor público,Secundário,Casado,Casa,41,0.578082,2.0
1389,F,True,True,1,Assalariado,Secundário,Casado,Casa,37,11.969863,3.0
1846,F,False,True,0,Assalariado,Secundário,Casado,Casa,56,6.353425,2.0
...,...,...,...,...,...,...,...,...,...,...,...
14422,F,False,True,0,Empresário,Secundário,Casado,Casa,28,0.375342,2.0
6262,F,True,False,1,Assalariado,Secundário,Casado,Casa,50,6.802740,3.0
6497,M,True,True,0,Assalariado,Secundário,União,Casa,29,1.306849,2.0
1036,M,False,True,0,Assalariado,Secundário,Separado,Casa,48,4.180822,1.0


In [128]:
X_train_b = pd.get_dummies(X_train_a, columns=['sexo', 'posse_de_veiculo', 'posse_de_imovel', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia'], drop_first=True)
X_train_b.head()

,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia,sexo_M,posse_de_veiculo_True,posse_de_imovel_True,tipo_renda_Bolsista,tipo_renda_Empresário,tipo_renda_Pensionista,...,educacao_Superior incompleto,estado_civil_Separado,estado_civil_Solteiro,estado_civil_União,estado_civil_Viúvo,tipo_residencia_Casa,tipo_residencia_Com os pais,tipo_residencia_Comunitário,tipo_residencia_Estúdio,tipo_residencia_Governamental
10766,0,50,5.509589,2.0,0,1,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
936,1,23,1.158904,3.0,1,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
695,0,41,0.578082,2.0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1389,1,37,11.969863,3.0,0,1,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1846,0,56,6.353425,2.0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [136]:
variaveis = stepwise_selection(X = X_train_b,y = y_train)

print('resulting features:')
print(variaveis)

Add  tempo_emprego                  with p-value 0.0
#############
['tempo_emprego']
Add  sexo_M                         with p-value 1.26513e-242
#############
['tempo_emprego', 'sexo_M']
Add  tipo_renda_Empresário          with p-value 4.12304e-06
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário']
Add  idade                          with p-value 2.37794e-05
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'idade']
Add  educacao_Superior completo     with p-value 0.00080423
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'idade', 'educacao_Superior completo']
Add  posse_de_imovel_True           with p-value 0.043736
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'idade', 'educacao_Superior completo', 'posse_de_imovel_True']
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'idade', 'educacao_Superior completo', 'posse_de_imovel_True']
resulting features:
['tempo_emprego', 'sexo_M', 'tipo_renda_Em

In [138]:
reg_stepwise = sm.OLS(y_train, sm.add_constant(pd.DataFrame(X_train_b[variaveis]))).fit()
reg_stepwise.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.246
Model:                            OLS   Adj. R-squared:                  0.245
Method:                 Least Squares   F-statistic:                     505.3
Date:                Sat, 16 Dec 2023   Prob (F-statistic):               0.00
Time:                        20:41:52   Log-Likelihood:                -97075.
No. Observations:                9320   AIC:                         1.942e+05
Df Residuals:                    9313   BIC:                         1.942e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                      -2878.6879    417.758     -6.891      0.000   -3697.586   -2059.790
tempo_emprego                571.0854     13.213     43.221      0.000     545.185     596.986
sexo_M                      6202.1282    177.260     34.989      0.000    5854.661    6549.596
tipo_renda_Empresário        836.5997    188.680      4.434      0.000     466.746    1206.453
idade                         40.9597      9.561      4.284      0.000      22.217      59.702
educacao_Superior completo   571.4215    173.693      3.290      0.001     230.944     911.899
posse_de_imovel_True         357.1090    177.060      2.017      0.044      10.034     704.184
==============================================================================
Omnibus:                    13578.919   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          8122349.469
Skew:                           8.601   Prob(JB):                         0.00
Kurtosis:                     146.597   Cond. No.                         218.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

O melhor modelo foi utilizando o métido Lasso com alpha = 0.1 obteve o maior R².

R² para alpha=0.1: 0.29801230507519516

In [156]:
# Defina os valores de alpha que você quer testar
alphas = [0.1, 0.15, 0.2,0.3,0.4,0.5]

# Normalizar as características
scaler = StandardScaler()
X_train_1_scaled = scaler.fit_transform(X_train_1)
X_test_1_scaled = scaler.transform(X_test_1)

# Loop sobre os valores de alpha
for alpha in alphas:
    # Criar um modelo de regressão LASSO
    lasso_model = Lasso(alpha=alpha)

    # Treinar o modelo com os dados de treinamento normalizados
    lasso_model.fit(X_train_1_scaled, y_train_1)

    # Fazer previsões nos dados de teste normalizados
    y_pred = lasso_model.predict(X_test_1_scaled)

    # Avaliar o R²
    r2 = r2_score(y_test_1, y_pred)

    # Exibir o resultado
    print(f'R^2 para alpha={alpha}: {r2}')

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.916e+09, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(


R^2 para alpha=0.1: 0.297982231838379


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.774e+09, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(


R^2 para alpha=0.15: 0.29798990251178636


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.758e+08, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(


R^2 para alpha=0.2: 0.29799753443595567


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.953e+08, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(


R^2 para alpha=0.3: 0.29801268203671205
R^2 para alpha=0.4: 0.29802767464063673
R^2 para alpha=0.5: 0.29804251224772804


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.966e+08, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.091e+08, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(


Utilizando um alpha de 0.5 foi possível aumentar o R² para 0.29804251224772804

In [159]:
# Criar um modelo de árvore de regressão
tree_model = DecisionTreeRegressor()

# Treinar o modelo com os dados de treinamento
tree_model.fit(X_train_1, y_train_1)

# Fazer previsões nos dados de teste
y_pred_tree = tree_model.predict(X_test_1)

# Avaliar o R²
r2_tree = r2_score(y_test_1, y_pred_tree)

# Exibir o resultado
print(f'R^2 para árvore de regressão: {r2_tree}')

R^2 para árvore de regressão: 0.23733888642968204


O R² da arvore de regressão ficou menor do que o do Lasso, portanto o modelo Lasso aparentemente continua sendo o melhor.